# El tour del caballo

Un problema muy célebre en el mundo de las matemáticas es el *Knight's Tour problem*.  Se trata de encontrar un camino recorrido por un caballo de ajedrez desde una casilla de comienzo de manera que recorre todas las casillas del tablero de ajedrez sin pasar dos veces por ninguna.

In [5]:
X,Y = 8,8
casillas = [(x,y) for x in range(X) for y in range(Y)]
movimientos_caballo = [ (1,2), (1,-2), (-1,2), (-1,-2), \
                        (2,1), (-2,1), (2,-1), (-2,-1) ]
def casillas_vecinas(p):
    x,y = p
    return [ (x+dx,y+dy) for dx,dy in movimientos_caballo \
                 if x+dx in range(X) and y+dy in range(Y) ]

vecinos = { p: casillas_vecinas(p) for p in casillas }

def caballo(inicial):
    return buscar_camino([inicial], set(casillas) - {inicial})
    
def buscar_camino(movimientos, pendientes):
    if len(pendientes) == 0:
        return movimientos

    def alcanzables(x):
        return [v for v in vecinos[x] if v in pendientes]
    
    def wandorf(x):
        return len(alcanzables(x))

    posibles = sorted(alcanzables(movimientos[-1]), key=wandorf)
    #posibles = alcanzables(movimientos[-1])
    for siguiente in posibles:
        intento_movimientos = movimientos.copy()
        intento_movimientos.append(siguiente)
        intento_pendientes = pendientes.copy()
        intento_pendientes.remove(siguiente)
        try:
            return buscar_camino(intento_movimientos, intento_pendientes)
        except ValueError:
            pass
    raise ValueError('Sin solución')

print(caballo((,0)))

[(0, 0), (1, 2), (2, 0), (0, 1), (1, 3), (0, 5), (1, 7), (3, 6), (5, 7), (7, 6), (6, 4), (7, 2), (6, 0), (4, 1), (6, 2), (7, 0), (5, 1), (3, 0), (1, 1), (0, 3), (1, 5), (0, 7), (2, 6), (4, 7), (6, 6), (7, 4), (5, 5), (6, 7), (7, 5), (6, 3), (7, 1), (5, 0), (3, 1), (1, 0), (2, 2), (4, 3), (2, 4), (3, 2), (4, 0), (5, 2), (7, 3), (6, 1), (5, 3), (4, 5), (3, 7), (1, 6), (0, 4), (2, 5), (0, 6), (2, 7), (4, 6), (3, 4), (4, 2), (5, 4), (3, 3), (2, 1), (0, 2), (1, 4), (3, 5), (2, 3), (4, 4), (5, 6), (7, 7), (6, 5)]
